In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import  matplotlib.pyplot as plt

In [2]:
from google.colab import drive          # this facilitates linking your colab with drive.
drive.mount('/content/drive/')          # mounting the drive.


Mounted at /content/drive/


In [5]:
!unzip -uq "/content/drive/MyDrive/project/face mask/archive" -d "/content/drive/MyDrive/project/face mask/archive"       # to unzip the zip file consiting of data sets.

In [13]:
import os   # facilates working with paths and directories

main_dir="/content/drive/MyDrive/project/face mask/archive/New Masks Dataset"
train_dir=os.path.join(main_dir,'Train')     # creating directories for all the different folders
test_dir=os.path.join(main_dir,'Test')
valid_dir=os.path.join(main_dir,'Validation')

train_mask_dir=os.path.join(train_dir, 'Mask')        # this used to trsin the model order to train the model
train_nomask_dir=os.path.join(train_dir, 'Non Mask')  


In [8]:
print(train_mask_dir)          # just to confirm that everything is done right.

/content/drive/MyDrive/project/face mask/archive/New Masks Datasets/Train/Mask


In [14]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range = 0.2,
                                 rotation_range = 40,
                                 horizontal_flip = True
                                 )             # providing some data generator rules
test_datagen=ImageDataGenerator(rescale=1./255) # not gonna use it for trainings
valid_datagen=ImageDataGenerator(rescale=1./255) # same

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size=32,
                                                    class_mode='binary'
                                                  )   # based on provided set of rules we uniform all of the images in the folder
test_generator = test_datagen.flow_from_directory(test_dir,
                                              target_size=(150,150),
                                              batch_size=32,       
                                              class_mode='binary'
                                            )
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                             target_size=(150,150),
                             batch_size=32,
                             class_mode='binary'
                             )

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [15]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [16]:
model = Sequential() # we need this model to image conversions

model.add(Conv2D(32,(3,3),padding='SAME',activation='relu', input_shape=(150,150,3))) # 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout (0.5))

model.add(Conv2D(64, (3,3),padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Dropout (0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu')) 
model.add(Dropout (0.5)) 
model.add(Dense(1,activation='sigmoid')) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 37, 37, 64)        0

In [17]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy']) 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
history = model.fit(train_generator, # we are comparing this processed set of data to the validation data set
                    epochs = 30, # epoches means periods or sessions
                    validation_data= valid_generator) # basically it is where the model is trained to identify masked faces

Epoch 1/30
19/19 [==============================] - 59s 3s/step - loss: 2.7826 - accuracy: 0.4967 - val_loss: 0.6932 - val_accuracy: 0.5033
Epoch 2/30
19/19 [==============================] - 49s 3s/step - loss: 0.6776 - accuracy: 0.5667 - val_loss: 0.6842 - val_accuracy: 0.5000
Epoch 3/30
19/19 [==============================] - 50s 3s/step - loss: 0.5995 - accuracy: 0.6817 - val_loss: 0.5799 - val_accuracy: 0.6863
Epoch 4/30
19/19 [==============================] - 51s 3s/step - loss: 0.4208 - accuracy: 0.8333 - val_loss: 0.3515 - val_accuracy: 0.8954
Epoch 5/30
19/19 [==============================] - 56s 3s/step - loss: 0.2792 - accuracy: 0.9067 - val_loss: 0.3468 - val_accuracy: 0.8529
Epoch 6/30
19/19 [==============================] - 51s 3s/step - loss: 0.2665 - accuracy: 0.9033 - val_loss: 0.2826 - val_accuracy: 0.9118
Epoch 7/30
19/19 [==============================] - 48s 3s/step - loss: 0.2413 - accuracy: 0.9167 - val_loss: 0.3085 - val_accuracy: 0.9118
Epoch 8/30
19/19 [==

In [19]:
test_loss, test_acc = model.evaluate(test_generator)
print('test aac :{} test loss:{}'.format(test_acc,test_loss))   #evaluating final accuracy and loss of the model

4/4 [==============================] - 3s 482ms/step - loss: 0.1617 - accuracy: 0.9100
test aac :0.9100000262260437 test loss:0.16165000200271606


In [31]:
import numpy as np 
from google.colab import files 
import keras.utils as image

uploaded=files.upload()    # in order upload the file
#print (uploaded)
for f in uploaded.keys():
    img_path= '/content/'+f  # for every image, image path will be saved as img_path
    img = image.load_img(img_path, target_size=(150,150)) 
    images=image.img_to_array(img) # this is used to convert into a numpy array
    images=np.expand_dims(images,axis=0) # 
    prediction=model.predict(images)
    if prediction==0:
        print(f, 'mask is present')
    else:
        print(f,' no mask is present')

Saving 2655.png to 2655 (1).png
1/1 [==============================] - 0s 55ms/step
2655.png mask is present


In [32]:
model.save('latestmodel.h5')